# Practing with _Think Bayes_ by Allen Downey

Using examples from the book, I break them down into digestable, transparent stories to fully understand bayesian thinking and analysis.

[Dice Problem](#dice)

In [2]:
def import_file(full_path_to_module):
    try:
        import os
        module_dir, module_file = os.path.split(full_path_to_module)
        module_name, module_ext = os.path.splitext(module_file)
        save_cwd = os.getcwd()
        os.chdir(module_dir)
        module_obj = __import__(module_name)
        module_obj.__file__ = full_path_to_module
        globals()[module_name] = module_obj
        os.chdir(save_cwd)
    except:
        raise ImportError

import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/thinkbayes.py')

import thinkbayes
import matplotlib.pyplot as plt

<a name="dice"></a>
## The dice problem

In [47]:
import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/dice.py')
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
import itertools as it

In the book, the main output for this problem is

In [39]:
import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/dice.py')
dice.main()

After one 6
4 0.0
6 0.392156862745
8 0.294117647059
12 0.196078431373
20 0.117647058824
After more rolls
4 0.0
6 0.0
8 0.915845271969
12 0.080403425797
20 0.00375130223399


In [40]:
import pandas as pd
data = 6
v = dice.Dice([4, 6, 8, 12, 20])
u = dice.Dice([4, 6, 8, 12, 20])
print("Hypotheses and priors")
print(u.d)
tmp = []
for hypo in u.Values():
    like = u.Likelihood(data, hypo)
    print("\n")
    print("If I rolled a "+repr(hypo)+"-sided die, how does my prior probability change if I roll an "+repr(data)+"?")
    print(u.d.get(hypo, 0) * like)
    tmp.append(u.d.get(hypo, 0) * like)
print('\n')
print('Make sure the probabilities sum up to 1 to be probabilities by definition!')
print('\n')
print('Sum of probabilities: '+repr(sum(tmp)))
print('\n')
print('Divide each probability by sum')
print("\n")
tmp2 = []
for t in tmp:
    tmp2.append(t / sum(tmp))
print(pd.Series(tmp2,index=hypos))
v.Update(6)
print('\nPosterior probabilities: probability of my hypotheses knowing I rolled an 6\n')
v.Print()

Hypotheses and priors
{8: 0.2, 12: 0.2, 4: 0.2, 6: 0.2, 20: 0.2}


If I rolled a 8-sided die, how does my prior probability change if I roll an 6?
0.025


If I rolled a 12-sided die, how does my prior probability change if I roll an 6?
0.0166666666667


If I rolled a 4-sided die, how does my prior probability change if I roll an 6?
0.0


If I rolled a 6-sided die, how does my prior probability change if I roll an 6?
0.0333333333333


If I rolled a 20-sided die, how does my prior probability change if I roll an 6?
0.01


Make sure the probabilities sum up to 1 to be probabilities by definition!


Sum of probabilities: 0.08500000000000002


Divide each probability by sum


4     0.294118
6     0.196078
8     0.000000
12    0.392157
20    0.117647
dtype: float64

Posterior probabilities: probability of my hypotheses knowing I rolled an 6

4 0.0
6 0.392156862745
8 0.294117647059
12 0.196078431373
20 0.117647058824


The take away from this is that the probability of the hypotheses given more data changes. But how is this done and how do we understand this? I'm going to do this example from scratch.

#### Set up

Suppose I have a box of die that contains a 4-sided die, a 6-sided due, a 8-sided die, a 12-sided die, and a 20-sided die.

Suppose I select a die from the box at random, roll it, and get a six. 

#### Question

What is the probability that I rolled each die if I got a 6?

The approach is to define my hypotheses, my data, and likelihood function.

#### Hypothesis

I rolled a 4, 6, 8, 12, or 20-sided die. For example, a 4-sided die is a hypothesis. You're are making a statement about the die you are rolling, which you want to test.

Here, we have 5 hypotheses. 

We initially assume that there's an equally likely chance of rolling a 4-sided die, a 6-sided die, etc. 

This initial assumption is called our **prior**. Because each of the N = 5 hypotheses are equally likely, each has a probability of 1 / N or 0.2 chance of being rolled.

In [41]:
hypos = [4,6,8,12,20]

priors = [0.2]

tmp = []
for x in it.product(hypos,priors):
    tmp.append(x)
    
print pd.DataFrame(tmp,columns=['H',"P(H)"])

    H  P(H)
0   4   0.2
1   6   0.2
2   8   0.2
3  12   0.2
4  20   0.2


Our data we have collected is a die roll that gives a 6

In [42]:
data = np.arange(1,21).tolist()
data

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Let's make a dataframe that puts our hypotheses, priors and data all together. 

In [43]:
lstoflsts = []

for hypo in hypos:
    for prior in priors:
        for d in data:
            lst = [hypo,prior,d]
            lstoflsts.append(lst)
            
df = pd.DataFrame(lstoflsts,columns=['H','P(H)','D'])
print df.shape
df.head()

(100, 3)


,H,P(H),D
0,4,0.2,1
1,4,0.2,2
2,4,0.2,3
3,4,0.2,4
4,4,0.2,5


I can also calculate the probability of my data, which is just 1/N where N is the length of my data.

In [44]:
np.repeat((1 / len(data)),len(df['D']))
1/20

0

The question is, for the data I've collected, what is the probability of rolling each die?

To determine the probability, I need to calculuate the likelihood.

The likelihood is just the raw frequency of occurrence.

For example, the likelihood of rolling a 3 from a 6 sided die is 1/6, 1/8 from a 8 sided die, and so on. 

Let's add the likelihood of the data given the hypotheses to our dataframe.

In [24]:
likelihood = []
df2 = df.copy()
for i in range(df.shape[0]):
    h = df.iloc[i]['H']
    d = df.iloc[i]['D']
    if h < d:
        likelihood.append(0)
    if h >= d:
        p = df.iloc[i]['P(H)']
        likelihood.append(1.0/h)

df2['P(D|H)'] = likelihood
df2.head()

,H,P(H),D,P(D),P(D|H)
0,4,0.2,1,0.05,0.25
1,4,0.2,2,0.10,0.25
2,4,0.2,3,0.15,0.25
3,4,0.2,4,0.20,0.25
4,4,0.2,5,0.25,0.00


Now we can derive what is the probability of the hypotheses given the data we observe. This is our **posterior**. You can think of this value as an _update_ to our prior. Our prior assigned a probability to each hypothesis being true, but was very generic. Our _posterior_ gives a better value for the hypothesis because it's influenced by what we actually observe.

In [26]:
num = (df2['P(H)'].values * df2['P(D|H)'].values)
denom = df2['P(D)']
posterior = num 
df2['P(H|D)'] = posterior.tolist()
df2.head()

,H,P(H),D,P(D),P(D|H),P(H|D)
0,4,0.2,1,0.05,0.25,0.05
1,4,0.2,2,0.10,0.25,0.05
2,4,0.2,3,0.15,0.25,0.05
3,4,0.2,4,0.20,0.25,0.05
4,4,0.2,5,0.25,0.00,0.00


Back to the original question, for getting a 6, what is the probability that I rolled a 4, 6, 8, 12, or 20 sided die?

In [10]:
df2.query(' D == 6 ')

,H,P(H),D,P(D),P(D|H),P(H|D)
5,4,0.2,6,0.05,0.000000,0.000000
25,6,0.2,6,0.05,0.166667,0.666667
45,8,0.2,6,0.05,0.125000,0.500000
65,12,0.2,6,0.05,0.083333,0.333333
85,20,0.2,6,0.05,0.050000,0.200000


You can see how the prior, P(H), changed to the posterior, P(H|D). 